<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/AskAPatient/Experiment%20(Change%20testing%20dataset)/Codes/askApatient_change_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
# base_dir = 'medical-concept-normalization'

In [0]:
# os.mkdir(base_dir)

In [0]:
# os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    820      0 --:--:-- --:--:-- --:--:--   820
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [11]:
train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
train.head()
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15612 entries, 0 to 15611
Data columns (total 2 columns):
0    15612 non-null int64
1    15612 non-null object
dtypes: int64(1), object(1)
memory usage: 244.0+ KB


In [0]:
train.head()

In [0]:
data_lm = TextLMDataBunch.from_csv(path, 'data_fine_tuning_language_model.csv')

In [0]:
valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)

In [0]:
valid.head()

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
data_lm.save()
data_clas.save()

In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [33]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.853851,4.167956,0.226499,04:08


In [34]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.975871,3.680264,0.300064,04:04


In [35]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.714983,3.652295,0.303827,04:05


In [36]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.674717,3.628957,0.307621,04:05


In [0]:
learn.save_encoder('ft_fold0_first')

In [39]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.319933,3.123680,0.466272,02:04


In [40]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.600766,2.664512,0.539645,02:47


In [41]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.933764,2.308774,0.634320,07:30


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.752825,2.004853,0.663905,07:06
1,1.479630,1.857999,0.694675,06:47


In [43]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.399459,1.734400,0.707692,06:52
1,1.130690,1.661654,0.721893,07:20


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.156082,1.626836,0.724260,06:50
1,0.925883,1.541834,0.757396,07:12


In [45]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.033367,1.526660,0.751479,07:22
1,0.825011,1.476555,0.760947,06:57


In [46]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.013895,1.532622,0.760947,06:58
1,0.762466,1.433347,0.773965,07:04


In [48]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.824823,1.482345,0.762130,07:25
1,0.858864,1.437610,0.769231,07:26
2,0.678810,1.397230,0.777515,07:16
3,0.581665,1.390066,0.785799,06:55


In [0]:
learn.save_encoder('ft_fold0_second')

In [0]:
test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)

In [0]:
test.head()

In [0]:
data_test = TextClasDataBunch.from_df(path, train, test, vocab=data_lm.train_ds.vocab, bs=32)

In [62]:
learn = text_classifier_learner(data_test,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('ft_fold0_second')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.602280,1.752537,0.710496,02:10
